In [ ]:
# !pip install pandas
# !pip install sklearn
# !pip install matplotlib
# !pip install tensorflow

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer

tf.data.experimental.enable_debug_mode()
tf.config.run_functions_eagerly(True)

In [2]:
tweets = pd.read_csv("sarcasm_dataset.csv")

tweets.head()

,Unnamed: 0,tweet,sarcastic,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# dop NaN columns
tweets = tweets.drop(tweets.columns[0], axis=1)

tweets.tail()

,tweet,sarcastic,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
862,yo @claires do yall do hysterectomies?,1,1.0,0.0,0.0,0.0,0.0,1.0
863,@JacobWohlReport Do I need to aquire a wife be...,1,1.0,0.0,0.0,0.0,0.0,1.0
864,I get a lot of boy who cried wolf vibes from t...,1,0.0,1.0,0.0,0.0,0.0,0.0
865,Update: holding hands with your mom and walkin...,1,1.0,0.0,0.0,0.0,0.0,0.0
866,"I might be rubbish at driving, and have a less...",1,1.0,0.0,0.0,0.0,0.0,0.0


# Binary Classification

In [4]:
# dropping unnecessary columns for binary classification
tweets1 = tweets.drop(['sarcasm', 'irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question'], axis=1)
tweets1.dropna(how='any', inplace=True)
tweets1.head()

,tweet,sarcasm
0,The only thing I got from college is a caffein...,0.0
1,I love it when professors draw a big question ...,1.0
2,Remember the hundred emails from companies whe...,0.0
3,Today my pop-pop told me I was not “forced” to...,1.0
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1.0


In [5]:
# Convert label from float to int
def transform_float_to_int(value):
    return int(value)

tweets1['sarcastic'] = tweets1.sarcastic.apply(transform_float_to_int)

In [6]:
# Split Tweet and Sarcastic Column
X = tweets1['tweet']
y = tweets1['sarcastic']

# Split Train & Test Data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=101, train_size=0.75)

# LSTM Model

In [7]:
max_features = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.25
maxlen = 500
batch_size = 32

In [8]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

In [9]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (650, 500)
X_test shape: (217, 500)


In [10]:
# LSTM Model with single layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         2560000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 193s 9s/step - loss: 0.5668 - accuracy: 0.8154 - val_loss: 0.4871 - val_accuracy: 0.8249


In [11]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [12]:
# LSTM Model with single layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 149s 7s/step - loss: 0.5726 - accuracy: 0.8031 - val_loss: 0.4755 - val_accuracy: 0.8249


In [13]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [14]:
# LSTM Model with single layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 114s 5s/step - loss: 0.5894 - accuracy: 0.7923 - val_loss: 0.5128 - val_accuracy: 0.8249


In [15]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [16]:
# LSTM Model with single layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 126s 6s/step - loss: 0.5853 - accuracy: 0.7969 - val_loss: 0.4679 - val_accuracy: 0.8249


In [17]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [18]:
# LSTM Model with Two layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_4 (LSTM)               (None, None, 128)         131584    
                                                                 
 dropout_4 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,823,297
Trainable params: 2,823,297
No

In [19]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [20]:
# LSTM Model with Two layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_6 (LSTM)               (None, None, 128)         131584    
                                                                 
 dropout_6 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_7 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,823,297
Trainable params: 2,823,297
No

In [21]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [22]:
# LSTM Model with Two layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_8 (LSTM)               (None, None, 128)         131584    
                                                                 
 dropout_8 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_9 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,823,297
Trainable params: 2,823,297
No

In [23]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [24]:
# LSTM Model with Two layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(LSTM(128))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_10 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_10 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_11 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,823,297
Trainable params: 2,823,297
No

In [25]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [26]:
# LSTM Model with Three layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_12 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_12 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_13 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_13 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_14 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_14 (Dropout)        (None, 128)              

In [27]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [28]:
# LSTM Model with Three layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_15 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_15 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_16 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_16 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_17 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_17 (Dropout)        (None, 128)              

In [29]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [30]:
# LSTM Model with Three layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_18 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_18 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_19 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_19 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_20 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_20 (Dropout)        (None, 128)             

In [31]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [32]:
# LSTM Model with Three layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(LSTM(128))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_21 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_21 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_22 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_22 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_23 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_23 (Dropout)        (None, 128)             

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

# BiLSTM Model

In [ ]:
# BiLSTM Model with single layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with single layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with single layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with single layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with Two layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with Two layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with Two layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with Two layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.8))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with Three layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with Three layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with Three layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
# BiLSTM Model with Three layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.8))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.8))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

# RNN Model

In [33]:
# RNN Model with single layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dropout_24 (Dropout)        (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,593,025
Trainable params: 2,593,025
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 16s 777ms/step - loss: 0.4901 - accuracy: 0.8169 - val_loss: 0.4640 - val_accuracy: 0.8249


In [34]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [35]:
# RNN Model with single layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dropout_25 (Dropout)        (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,593,025
Trainable params: 2,593,025
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 16s 785ms/step - loss: 0.5033 - accuracy: 0.8108 - val_loss: 0.4667 - val_accuracy: 0.8249


In [36]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [37]:
# RNN Model with single layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dropout_26 (Dropout)        (None, 128)               0         
                                                                 
 dense_14 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,593,025
Trainable params: 2,593,025
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 16s 772ms/step - loss: 0.5078 - accuracy: 0.8000 - val_loss: 0.4646 - val_accuracy: 0.8249


In [38]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [39]:
# RNN Model with single layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dropout_27 (Dropout)        (None, 128)               0         
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,593,025
Trainable params: 2,593,025
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 16s 773ms/step - loss: 0.5904 - accuracy: 0.7446 - val_loss: 0.5264 - val_accuracy: 0.8249


In [40]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [41]:
# RNN Model with Two layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, None, 128)         32896     
                                                                 
 dropout_28 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dropout_29 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,625,921
Trainable params: 2,625,921
N

In [42]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [43]:
# RNN Model with Two layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(SimpleRNN(128))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, None, 128)         32896     
                                                                 
 dropout_30 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dropout_31 (Dropout)        (None, 128)               0         
                                                                 
 dense_17 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,625,921
Trainable params: 2,625,921
N

In [44]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [45]:
# RNN Model with Two layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(SimpleRNN(128))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, None, 128)         32896     
                                                                 
 dropout_32 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dropout_33 (Dropout)        (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,625,921
Trainable params: 2,625,921
N

In [46]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [47]:
# RNN Model with Two layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(SimpleRNN(128))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_34 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_35 (Dropout)        (None, 128)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,625,921
Trainable params: 2,625,921
N

In [48]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [49]:
# RNN Model with Three layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_12 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_36 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_37 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_14 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_38 (Dropout)        (None, 128)             

In [50]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [51]:
# RNN Model with Three layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(SimpleRNN(128))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_15 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_39 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_16 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_40 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_17 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_41 (Dropout)        (None, 128)             

In [52]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [53]:
# RNN Model with Three layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(SimpleRNN(128))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_18 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_42 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_19 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_43 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_20 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_44 (Dropout)        (None, 128)             

In [54]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [55]:
# RNN Model with Three layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(SimpleRNN(128))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_21 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_45 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_22 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_46 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_23 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_47 (Dropout)        (None, 128)             

In [56]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



# GRU Model

In [57]:
# GRU Model with Single layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru (GRU)                   (None, 128)               98688     
                                                                 
 dropout_48 (Dropout)        (None, 128)               0         
                                                                 
 dense_24 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,658,817
Trainable params: 2,658,817
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 120s 6s/step - loss: 0.6090 - accuracy: 0.8062 - val_loss: 0.4704 - val_accuracy: 0.8249


In [58]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [59]:
# GRU Model with Single layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_1 (GRU)                 (None, 128)               98688     
                                                                 
 dropout_49 (Dropout)        (None, 128)               0         
                                                                 
 dense_25 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,658,817
Trainable params: 2,658,817
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 118s 6s/step - loss: 0.5997 - accuracy: 0.8015 - val_loss: 0.4686 - val_accuracy: 0.8249


In [60]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [61]:
# GRU Model with Single layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_26 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_2 (GRU)                 (None, 128)               98688     
                                                                 
 dropout_50 (Dropout)        (None, 128)               0         
                                                                 
 dense_26 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,658,817
Trainable params: 2,658,817
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 119s 6s/step - loss: 0.6163 - accuracy: 0.7892 - val_loss: 0.4798 - val_accuracy: 0.8249


In [62]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [63]:
# GRU Model with Single layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_27 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_3 (GRU)                 (None, 128)               98688     
                                                                 
 dropout_51 (Dropout)        (None, 128)               0         
                                                                 
 dense_27 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,658,817
Trainable params: 2,658,817
Non-trainable params: 0
_________________________________________________________________
None
21/21 [==============================] - 119s 6s/step - loss: 0.6185 - accuracy: 0.7754 - val_loss: 0.4837 - val_accuracy: 0.8249


In [64]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [65]:
# GRU Model with Two layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_28 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_4 (GRU)                 (None, None, 128)         98688     
                                                                 
 dropout_52 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_5 (GRU)                 (None, 128)               98688     
                                                                 
 dropout_53 (Dropout)        (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,757,505
Trainable params: 2,757,505
N

In [66]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [67]:
# GRU Model with Two layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(GRU(128))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_29 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_6 (GRU)                 (None, None, 128)         98688     
                                                                 
 dropout_54 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_7 (GRU)                 (None, 128)               98688     
                                                                 
 dropout_55 (Dropout)        (None, 128)               0         
                                                                 
 dense_29 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,757,505
Trainable params: 2,757,505
N

In [68]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [69]:
# GRU Model with Two layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(GRU(128))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_30 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_8 (GRU)                 (None, None, 128)         98688     
                                                                 
 dropout_56 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_9 (GRU)                 (None, 128)               98688     
                                                                 
 dropout_57 (Dropout)        (None, 128)               0         
                                                                 
 dense_30 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,757,505
Trainable params: 2,757,505
N

In [70]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [71]:
# GRU Model with Two layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(GRU(128))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_31 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_10 (GRU)                (None, None, 128)         98688     
                                                                 
 dropout_58 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_11 (GRU)                (None, 128)               98688     
                                                                 
 dropout_59 (Dropout)        (None, 128)               0         
                                                                 
 dense_31 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,757,505
Trainable params: 2,757,505
N

In [72]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [73]:
# GRU Model with Three layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_32 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_12 (GRU)                (None, None, 128)         98688     
                                                                 
 dropout_60 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_13 (GRU)                (None, None, 128)         98688     
                                                                 
 dropout_61 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_14 (GRU)                (None, 128)               98688     
                                                                 
 dropout_62 (Dropout)        (None, 128)             

In [74]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [75]:
# GRU Model with Three layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(GRU(128))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_33 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_15 (GRU)                (None, None, 128)         98688     
                                                                 
 dropout_63 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_16 (GRU)                (None, None, 128)         98688     
                                                                 
 dropout_64 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_17 (GRU)                (None, 128)               98688     
                                                                 
 dropout_65 (Dropout)        (None, 128)             

In [76]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [77]:
# GRU Model with Three layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(GRU(128))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_34 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_18 (GRU)                (None, None, 128)         98688     
                                                                 
 dropout_66 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_19 (GRU)                (None, None, 128)         98688     
                                                                 
 dropout_67 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_20 (GRU)                (None, 128)               98688     
                                                                 
 dropout_68 (Dropout)        (None, 128)             

In [78]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



In [79]:
# GRU Model with Three layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(GRU(128))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_35 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_21 (GRU)                (None, None, 128)         98688     
                                                                 
 dropout_69 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_22 (GRU)                (None, None, 128)         98688     
                                                                 
 dropout_70 (Dropout)        (None, None, 128)         0         
                                                                 
 gru_23 (GRU)                (None, 128)               98688     
                                                                 
 dropout_71 (Dropout)        (None, 128)             

In [80]:
preds = model.predict(X_test, verbose=0)
predictions = [1 if x > 0.5 else 0 for x in preds]
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.82      1.00      0.90       179

    accuracy                           0.82       217
   macro avg       0.41      0.50      0.45       217
weighted avg       0.68      0.82      0.75       217



# Preparing Data for multi class classification


In [81]:
tweets_multi = tweets.copy()
tweets_multi = tweets_multi[tweets.sarcastic == 1]
tweets_multi.head()

,tweet,sarcastic,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
1,I love it when professors draw a big question ...,1,1.0,0.0,0.0,0.0,0.0,0.0
3,Today my pop-pop told me I was not “forced” to...,1,1.0,0.0,0.0,0.0,0.0,0.0
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,1.0,0.0,0.0,0.0,0.0,0.0
7,someone hit me w a horse tranquilizer istg ive...,1,1.0,0.0,0.0,0.0,0.0,0.0
8,Loving season 4 of trump does America. Funnies...,1,1.0,0.0,0.0,0.0,0.0,0.0


In [82]:
def convert_col_to_label(row):
    if row.irony == 1.0:
        return 'irony'
    if row.satire == 1.0:
        return 'satire';
    if row.understatement == 1.0:
        return 'understatement'
    if row.overstatement == 1.0:
        return 'overstatement'
    if row.rhetorical_question == 1.0:
        return 'rhetorical_question'
    return 'Other'

tweets_multi['label'] = tweets_multi.apply(lambda row: convert_col_to_label(row), axis=1)

In [83]:
tweets_multi = tweets_multi.drop(['sarcasm', 'sarcastic', 'irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question'], axis=1)

In [84]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
tweets_multi['label'] = labelencoder.fit_transform(tweets_multi['label'])

In [85]:
X = tweets_multi['tweet']
y = tweets_multi['label']

In [86]:
# Split Train & Test Data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=101, train_size=0.75)

# Deep Learning Multi Class Classification

In [87]:
max_features = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.25
maxlen = 500
batch_size = 32
classes_len = len(np.unique(y_train))

In [88]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

In [89]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (534, 500)
X_test shape: (179, 500)


# RNN Model

In [90]:
# RNN Model with Single layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_36 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_24 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_72 (Dropout)        (None, 128)               0         
                                                                 
 dense_36 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,593,670
Trainable params: 2,593,670
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 17s 996ms/step - loss: 0.9700 - accuracy: 0.7434 - val_loss: 0.7325 - val_accuracy: 0.7877


In [91]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [92]:
# RNN Model with Single layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_37 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_25 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_73 (Dropout)        (None, 128)               0         
                                                                 
 dense_37 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,593,670
Trainable params: 2,593,670
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 16s 948ms/step - loss: 0.9634 - accuracy: 0.7603 - val_loss: 0.7372 - val_accuracy: 0.7877


In [93]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [94]:
# RNN Model with Single layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_38 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_26 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_74 (Dropout)        (None, 128)               0         
                                                                 
 dense_38 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,593,670
Trainable params: 2,593,670
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 17s 992ms/step - loss: 1.0336 - accuracy: 0.7079 - val_loss: 0.9229 - val_accuracy: 0.7877


In [95]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [96]:
# RNN Model with Single layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_39 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_27 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_75 (Dropout)        (None, 128)               0         
                                                                 
 dense_39 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,593,670
Trainable params: 2,593,670
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 16s 940ms/step - loss: 1.1451 - accuracy: 0.6011 - val_loss: 0.7519 - val_accuracy: 0.7877


In [97]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [98]:
# RNN Model with Two layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_40 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_28 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_76 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_29 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_77 (Dropout)        (None, 128)               0         
                                                                 
 dense_40 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,626,566
Trainable params: 2,626,566
N

In [99]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [100]:
# RNN Model with Two layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(SimpleRNN(128))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_41 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_30 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_78 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_31 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_79 (Dropout)        (None, 128)               0         
                                                                 
 dense_41 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,626,566
Trainable params: 2,626,566
N

In [101]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [102]:
# RNN Model with Two layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(SimpleRNN(128))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_42 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_32 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_80 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_33 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_81 (Dropout)        (None, 128)               0         
                                                                 
 dense_42 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,626,566
Trainable params: 2,626,566
N

In [103]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [104]:
# RNN Model with Two layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(SimpleRNN(128))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_43 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_34 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_82 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_35 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_83 (Dropout)        (None, 128)               0         
                                                                 
 dense_43 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,626,566
Trainable params: 2,626,566
N

In [105]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [106]:
# RNN Model with Three layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_44 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_36 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_84 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_37 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_85 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_38 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_86 (Dropout)        (None, 128)             

In [107]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [108]:
# RNN Model with Three layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(SimpleRNN(128))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_45 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_39 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_87 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_40 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_88 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_41 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_89 (Dropout)        (None, 128)             

In [109]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [110]:
# RNN Model with Three layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(SimpleRNN(128))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_46 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_42 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_90 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_43 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_91 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_44 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_92 (Dropout)        (None, 128)             

In [111]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [112]:
# RNN Model with Three layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(SimpleRNN(128))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_47 (Embedding)    (None, None, 128)         2560000   
                                                                 
 simple_rnn_45 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_93 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_46 (SimpleRNN)   (None, None, 128)         32896     
                                                                 
 dropout_94 (Dropout)        (None, None, 128)         0         
                                                                 
 simple_rnn_47 (SimpleRNN)   (None, 128)               32896     
                                                                 
 dropout_95 (Dropout)        (None, 128)             

In [113]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.78      0.94      0.85       141
           3       0.10      0.04      0.05        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.74      0.74      0.74       179
   macro avg       0.15      0.16      0.15       179
weighted avg       0.63      0.74      0.68       179



# LSTM Model

In [114]:
# LSTM Model with Single layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_48 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_24 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_96 (Dropout)        (None, 128)               0         
                                                                 
 dense_48 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,692,358
Trainable params: 2,692,358
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 104s 6s/step - loss: 1.2898 - accuracy: 0.7360 - val_loss: 0.7888 - val_accuracy: 0.7877


In [115]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [116]:
# LSTM Model with Single layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_49 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_25 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_97 (Dropout)        (None, 128)               0         
                                                                 
 dense_49 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,692,358
Trainable params: 2,692,358
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 100s 6s/step - loss: 1.2985 - accuracy: 0.7491 - val_loss: 0.8567 - val_accuracy: 0.7877


In [117]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [118]:
# LSTM Model with Single layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_50 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_26 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_98 (Dropout)        (None, 128)               0         
                                                                 
 dense_50 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,692,358
Trainable params: 2,692,358
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 123s 7s/step - loss: 1.3916 - accuracy: 0.7434 - val_loss: 0.7491 - val_accuracy: 0.7877


In [119]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [120]:
# LSTM Model with Single layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_51"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_51 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_27 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_99 (Dropout)        (None, 128)               0         
                                                                 
 dense_51 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,692,358
Trainable params: 2,692,358
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 109s 6s/step - loss: 1.4065 - accuracy: 0.6854 - val_loss: 0.9057 - val_accuracy: 0.7877


In [121]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [122]:
# LSTM Model with Two layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(LSTM(128))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_52"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_52 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_28 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_100 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_29 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_101 (Dropout)       (None, 128)               0         
                                                                 
 dense_52 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,823,942
Trainable params: 2,823,942
N

In [123]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [124]:
# LSTM Model with Two layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_53 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_30 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_102 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_31 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_103 (Dropout)       (None, 128)               0         
                                                                 
 dense_53 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,823,942
Trainable params: 2,823,942
N

In [125]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [126]:
# LSTM Model with Two layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_54 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_32 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_104 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_33 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_105 (Dropout)       (None, 128)               0         
                                                                 
 dense_54 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,823,942
Trainable params: 2,823,942
N

In [127]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [128]:
# LSTM Model with Two layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_55 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_34 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_106 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_35 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_107 (Dropout)       (None, 128)               0         
                                                                 
 dense_55 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,823,942
Trainable params: 2,823,942
N

In [129]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [130]:
# LSTM Model with Three layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_56 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_36 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_108 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_37 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_109 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_38 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_110 (Dropout)       (None, 128)             

In [131]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [132]:
# LSTM Model with Three layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_57 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_39 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_111 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_40 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_112 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_41 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_113 (Dropout)       (None, 128)             

In [133]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [134]:
# LSTM Model with Three layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_58"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_58 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_42 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_114 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_43 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_115 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_44 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_116 (Dropout)       (None, 128)             

In [135]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [136]:
# LSTM Model with Three layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(LSTM(128))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_59"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_59 (Embedding)    (None, None, 128)         2560000   
                                                                 
 lstm_45 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_117 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_46 (LSTM)              (None, None, 128)         131584    
                                                                 
 dropout_118 (Dropout)       (None, None, 128)         0         
                                                                 
 lstm_47 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_119 (Dropout)       (None, 128)             

In [137]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



# GRU Model

In [138]:
# GRU Model with Single layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_60 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_24 (GRU)                (None, 128)               98688     
                                                                 
 dropout_120 (Dropout)       (None, 128)               0         
                                                                 
 dense_60 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,659,462
Trainable params: 2,659,462
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 121s 7s/step - loss: 1.4875 - accuracy: 0.7378 - val_loss: 1.1906 - val_accuracy: 0.7877


In [139]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [140]:
# GRU Model with Single layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_61 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_25 (GRU)                (None, 128)               98688     
                                                                 
 dropout_121 (Dropout)       (None, 128)               0         
                                                                 
 dense_61 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,659,462
Trainable params: 2,659,462
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 120s 7s/step - loss: 1.5636 - accuracy: 0.7154 - val_loss: 0.8432 - val_accuracy: 0.7877


In [141]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       141
           3       0.00      0.00      0.00        28
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5

   micro avg       0.79      0.79      0.79       179
   macro avg       0.13      0.17      0.15       179
weighted avg       0.62      0.79      0.69       179



In [142]:
# GRU Model with Single layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

Model: "sequential_62"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_62 (Embedding)    (None, None, 128)         2560000   
                                                                 
 gru_26 (GRU)                (None, 128)               98688     
                                                                 
 dropout_122 (Dropout)       (None, 128)               0         
                                                                 
 dense_62 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,659,462
Trainable params: 2,659,462
Non-trainable params: 0
_________________________________________________________________
None
 5/17 [=======>......................] - ETA: 1:14 - loss: 1.7694 - accuracy: 0.4938

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# GRU Model with Single layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# GRU Model with Two layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# GRU Model with Two layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(GRU(128))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# GRU Model with Two layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(GRU(128))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# GRU Model with Two layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(GRU(128))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# GRU Model with Three layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# GRU Model with Three layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(GRU(128))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# GRU Model with Three layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(GRU(128))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# GRU Model with Three layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(GRU(128))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

# BiLSTM Model

In [ ]:
# LSTM Model with Single layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Single layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Single layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Single layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Two layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Two layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Two layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Two layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.8))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Three layer and 0.2 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Three layer and 0.4 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.4))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Three layer and 0.6 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.6))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))

In [ ]:
# LSTM Model with Three layer and 0.8 dropout rate
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.8))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.8))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.8))
model.add(Dense(classes_len, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(X_train)
print(model.summary())

model.fit(X_train, y_train, batch_size=batch_size, epochs=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test, verbose=0)
predictions = [np.argmax(x) for x in preds]
print(classification_report(y_test, predictions, zero_division=0, labels=tweets_multi.label.unique()))